In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import re
from IPython.display import display, HTML
import seaborn as sns

In [2]:
SHOW_FIGURES = False

In [3]:
if not SHOW_FIGURES:
    matplotlib.use("pgf")
    matplotlib.rcParams.update({
        "pgf.texsystem": "pdflatex",
        'font.family': 'serif',
        'text.usetex': True,
        'pgf.rcfonts': False,
    })

In [4]:
file_paths = ['../results_Park.csv', # more than 10
              '../results_ShortestQueue_Random.csv', # check
              '../results_Hungarian_140.csv', # check
              '../results_CP_200.csv', # check
              '../results_kBatch_8.csv', # done by tomorrow
              '../results_RoundRobin.csv' # only 9
             ]

In [5]:
csv_dict = []
def load_file(file_path, csv_dict):
    with open(file_path, 'r') as file:
        # Iterate over each line in the fileplt.plot(x_fit, y_fit, color='green', label='Polynomial Approximation')
        for line in file:
            # Split the line content by comma
            line_data = line.strip().split(',')
            if len(line_data) == 19:
                nineteen_keys = ['method', 'start time', 'end time', 'real execution time', 'system execution time',
                                 'delta', 'average make span', 'result message',
                                'makespan', 'resource occupation', 'resource fairness', 'weighted resource fairness',
                                'assigned tasks', 'allocated tasks', 'postponed tasks',
                                'optimal solution', 'feasible solution', 'no solution',
                                'resource worklist rule'
                                ]
                row_dict = dict(zip(nineteen_keys, line_data))
                for k in ['optimal solution', 'feasible solution', 'no solution']:
                    if row_dict[k] == '':
                        row_dict[k] = 0
                csv_dict.append(row_dict)
            else:
                print(len(line_data))

df_dtypes = {'method' : str, 'start time' : str, 'end time' : str, 'real execution time' : float, 'system execution time' : float,
             'delta' : float, 'makespan' : float, 'result message' : str,
             'makespan' : float, 'resource occupation' : float, 'resource fairness' : float, 'weighted resource fairness' : float,
             'assigned tasks' : int, 'allocated tasks' : int, 'postponed tasks' : int,
             'optimal solution' : int, 'feasible solution' : int, 'no solution' : int,
             'resource worklist rule' : str}

for f in file_paths:
    load_file(f, csv_dict)

df = pd.DataFrame(csv_dict)
for k, v in df_dtypes.items():
    df[k] = df[k].astype(v)

In [6]:
pd.option_context('display.max_rows', None, 'display.max_columns', None)  # more options can be specified also
display(HTML(df.to_html()))

,method,start time,end time,real execution time,system execution time,delta,average make span,result message,makespan,resource occupation,resource fairness,weighted resource fairness,assigned tasks,allocated tasks,postponed tasks,optimal solution,feasible solution,no solution,resource worklist rule
0,Park,2024-03-03 12:09:10,2024-03-03 13:21:39,4349.583230,112179.797090,0.900,17.206706601432195,COMPLETED: you completed 1680 hours of simulated customer cases. 9692 cases started. 9606 cases run to completion.,17.360909,0.785048,0.031043,1.180851,77353,417082,339729,0,0,0,
1,Park,2024-03-03 13:21:40,2024-03-03 14:26:34,3894.029671,101771.082179,0.920,17.914925297844352,COMPLETED: you completed 1680 hours of simulated customer cases. 8914 cases started. 8828 cases run to completion.,18.089454,0.736602,0.037313,1.372683,71239,388163,316924,0,0,0,
2,Park,2024-03-03 14:26:34,2024-03-03 15:50:40,5046.330946,118384.914716,0.940,21.65121742881701,COMPLETED: you completed 1680 hours of simulated customer cases. 9642 cases started. 9567 cases run to completion.,21.821299,0.788288,0.028515,1.451327,78160,423634,345474,0,0,0,
3,Park,2024-03-03 15:50:41,2024-03-03 17:16:27,5146.126878,123560.311336,0.960,26.33545111669285,COMPLETED: you completed 1680 hours of simulated customer cases. 9592 cases started. 9498 cases run to completion.,26.596216,0.785307,0.024519,1.228184,77179,425663,348484,0,0,0,
4,Park,2024-03-03 17:16:27,2024-03-03 18:24:21,4073.929705,105919.429758,0.980,17.340833169503785,COMPLETED: you completed 1680 hours of simulated customer cases. 9095 cases started. 9044 cases run to completion.,17.438682,0.770744,0.028156,1.819650,73967,408498,334531,0,0,0,
5,Park,2024-03-03 18:24:21,2024-03-03 19:25:41,3679.928840,98719.385536,1.000,16.691193729852863,COMPLETED: you completed 1680 hours of simulated customer cases. 8854 cases started. 8748 cases run to completion.,16.893852,0.754244,0.029603,1.620962,71434,390740,319306,0,0,0,
6,Park,2024-03-03 19:25:41,2024-03-03 20:36:45,4263.831167,109776.709924,1.020,20.022503396888556,COMPLETED: you completed 1680 hours of simulated customer cases. 9142 cases started. 8949 cases run to completion.,20.455760,0.809481,0.027550,1.550476,73022,407400,334378,0,0,0,
7,Park,2024-03-03 20:36:46,2024-03-03 22:35:04,7098.598126,144091.326758,1.040,34.15538986395024,COMPLETED: you completed 1680 hours of simulated customer cases. 10247 cases started. 10103 cases run to completion.,34.642691,0.829213,0.024717,1.555958,81942,451680,369738,0,0,0,
8,Park,2024-03-03 22:35:05,2024-03-03 23:37:02,3716.821540,98033.941546,1.060,16.670707766981696,COMPLETED: you completed 1680 hours of simulated customer cases. 8923 cases started. 8838 cases run to completion.,16.831398,0.757237,0.041985,1.510468,71760,388407,316647,0,0,0,
9,Park,2024-03-03 23:37:02,2024-03-04 00:40:17,3795.386204,100098.031292,1.080,19.039583477283553,COMPLETED: you completed 1680 hours of simulated customer cases. 8731 cases started. 8656 cases run to completion.,19.204631,0.765968,0.029422,1.526738,70239,382487,312248,0,0,0,


In [7]:
# When more than 10 delta values are present sample 10 random ones
result = df.groupby(['method']).apply(lambda x: x.sample(9)).reset_index(drop=True)
#result
df = result
print(len(df))

63


/tmp/ipykernel_15568/1047646439.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby(['method']).apply(lambda x: x.sample(9)).reset_index(drop=True)


In [8]:

park_df = df[(df['method'] == 'Park')]
sq_df = df[(df['method'] == 'ShortestQueue')]
rr_df = df[(df['method'] == 'RoundRobin')]
ra_df = df[(df['method'] == 'Random')]
kb_df = df[(df['method'] == 'KBatch')]


our_upm_df = df[(df['method'] == 'MILP')]
our_ta_df = df[(df['method'] == 'Hungarian')]


def make_boxplot(y, label=None):
    fig = plt.gcf()
    
    fig,ax = plt.subplots(1)
    fig.set_size_inches(6, 2.5)

    sns.boxplot(y=y, x='method', data=our_upm_df, ax=ax, showfliers=False) 
    sns.boxplot(y=y, x='method', data=our_ta_df, ax=ax, showfliers=False) 
    sns.boxplot(y=y, x='method', data=park_df, ax=ax, showfliers=False)
    sns.boxplot(y=y, x='method', data=kb_df, ax=ax, showfliers=False)
    sns.boxplot(y=y, x='method', data=sq_df, ax=ax, showfliers=False) 
    sns.boxplot(y=y, x='method', data=rr_df, ax=ax, showfliers=False) 
    sns.boxplot(y=y, x='method', data=ra_df, ax=ax, showfliers=False) 

    ax.set(xlabel=None)
    ax.set_xticks([0,1,2,3,4,5,6], ['PMS\n$\delta=2.0$\nfastest', 'TA \n$\delta=1.4$', 'Park \&\nSong', 'k-Batch\nk=8', 'R-SHQ',
                               'R-RRA', 'R-RMA'])
    if label:
        ax.set(ylabel=label)
    else:
        ax.set(ylabel=y.capitalize())
    
    plt.tight_layout(pad=0.2)
    if SHOW_FIGURES:
        plt.show() 
    else:
        plt.savefig('boxplot_' + y + '.pgf')

make_boxplot('makespan', 'Avg. cycle time')
make_boxplot('resource occupation')
make_boxplot('resource fairness')
make_boxplot('weighted resource fairness')




In [9]:
def make_boxplot(y, label=None):
    fig = plt.gcf()
    
    fig,ax = plt.subplots(1)
    fig.set_size_inches(6, 2.5)
    sns.boxplot(y=y, x='method', data=our_upm_df, ax=ax, showfliers=False) 
    sns.boxplot(y=y, x='method', data=our_ta_df, ax=ax, showfliers=False) 
    sns.boxplot(y=y, x='method', data=park_df, ax=ax, showfliers=False) 


    ax.set(xlabel=None)
    ax.set_xticks([0,1,2], ['PMS\n$\delta=2.0$\nfastest', 'TA\n$\delta=1.4$', 'Park \& Song'])
    if label:
        ax.set(ylabel=label)
    else:
        ax.set(ylabel=y.capitalize())
    
    plt.tight_layout(pad=0.2)
    if SHOW_FIGURES:
        plt.show() 
    else:
        plt.savefig('boxplot_3_' + y + '.pgf')

make_boxplot('makespan', 'Avg. cycle time')
make_boxplot('resource occupation')
make_boxplot('resource fairness')
make_boxplot('weighted resource fairness')

In [10]:
print(llqp_df)

NameError: name 'llqp_df' is not defined